In [11]:
# Import necessary modules
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

Download csv file from 'FED parsing.ipynb' file

In [5]:
speech_data=pd.read_csv("df_min_press_speech.csv")
speech_data.head()

,date,text
0,2012-01-06,"It is the start of a new year, the traditional..."
1,2012-01-06,\r\n Thank you for the opportunity to sp...
2,2012-01-07,\r\n Thank you and happy New Year. It is...
3,2012-01-13,\r\n It's a pleasure to be here this mor...
4,2012-01-16,\r\n It is certainly a pleasure to be he...


### Create function that generates embeddings from each text input

In [6]:
# Load FinBERT tokenizer and model
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert_model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

# Function to generate embeddings for given text input
def generate_text_embedding(input_text):
    # Tokenize the input text and prepare tensors
    tokenized_inputs = finbert_tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Disable gradient calculation for inference
    with torch.no_grad():
        # Pass tokenized inputs through the model
        model_outputs = finbert_model(**tokenized_inputs)
    
    # Extract the last hidden state from the model outputs
    hidden_state = model_outputs.last_hidden_state
    
    # Calculate the mean of the hidden states for a single sentence embedding
    avg_sentence_embedding = torch.mean(hidden_state, dim=1)
    
    # Detach the embedding from the computation graph and convert to NumPy
    return avg_sentence_embedding.detach().cpu().numpy()


### Create 100 Hawkish and dovish phrases from GPT

In [7]:
hawkish_phrases = [
    "Raise interest rates",
    "Interest rate hike",
    "Tightening monetary policy",
    "Monetary tightening",
    "Combat inflation",
    "Inflationary pressures",
    "Curb inflation",
    "Control inflation",
    "Restrictive monetary policy",
    "Aggressive policy stance",
    "Reduce liquidity",
    "Tight fiscal policy",
    "Reining in inflation",
    "Price stability",
    "Reduce inflation",
    "Inflation expectations",
    "Preemptive tightening",
    "Increase rates",
    "Rate normalization",
    "Policy tightening",
    "Persistent inflation",
    "Rising inflation",
    "Higher inflation",
    "Elevated inflation",
    "Tighten conditions",
    "Raise borrowing costs",
    "Control price pressures",
    "Price hikes",
    "Inflationary expectations",
    "Price controls",
    "Elevated price levels",
    "Fight inflation",
    "Higher inflation targets",
    "Mitigate inflation",
    "Anchor inflation expectations",
    "Reducing stimulus",
    "Contracting monetary policy",
    "Liquidity squeeze",
    "Monetary restraint",
    "Curb excessive inflation",
    "Slow economic overheating",
    "Preemptive measures",
    "Macroprudential tightening",
    "Inflationary trend",
    "Repricing inflation risk",
    "Diminish demand",
    "Moderate demand",
    "Tightening cycle",
    "Contractionary policy",
    "Monetary discipline",
    "Interest rate increase",
    "Reducing asset purchases",
    "Rate hikes",
    "Constrain economic growth",
    "Reduce demand-side pressures",
    "Inflation overshoot",
    "Persistent price pressures",
    "Tighter financial conditions",
    "Counter inflation",
    "Normalization of policy",
    "Reduce bond buying",
    "Policy rate increase",
    "Slowing down asset purchases",
    "Rate hike guidance",
    "Slowing inflation",
    "Rising bond yields",
    "Defensive tightening",
    "Proactive policy stance",
    "Tightening stance",
    "Reduce balance sheet",
    "Ending quantitative easing",
    "Shrinking the balance sheet",
    "Higher interest rates",
    "Price control efforts",
    "Control inflationary spirals",
    "Curb wage inflation",
    "Inflationary spike",
    "Interest rate pressure",
    "Withdrawing stimulus",
    "Liquidity withdrawal",
    "Prevent overheating",
    "Constrain price growth",
    "Slow rising prices",
    "Tighten credit conditions",
    "Reduce money supply",
    "Raise financing costs",
    "Strengthen inflation control",
    "Pursuing rate hikes",
    "Dampen inflation",
    "Adjust rates upward",
    "Shift toward tightening",
    "Suppress demand-driven inflation",
    "Aggressive tightening",
    "Contractionary bias",
    "Discourage borrowing",
    "Strengthen inflation expectations",
    "Address inflationary threats",
    "Preemptive rate increases",
    "Interest rate hikes anticipated",
    "Move to tighten policy",
    "The persistent inflationary pressures necessitate a more aggressive monetary tightening than previously forecasted.",
    "To counterbalance the overheating economy, the central bank is committed to a series of rate hikes over the coming quarters.",
    "Rising inflation expectations demand immediate corrective measures, including the accelerated normalization of interest rates.",
    "The combination of strong wage growth and elevated demand signals the need for restrictive monetary policy.",
    "Policymakers are prepared to implement successive interest rate increases to prevent inflation from becoming entrenched.",
    "The economy's robust expansion justifies the continuation of a contractionary monetary stance to rein in overheating risks.",
    "Ongoing inflationary pressures have prompted the need for a higher and more prolonged path of interest rates.",
    "Labor market tightness, coupled with rising inflationary trends, underscores the necessity of immediate policy tightening.",
    "Participants emphasized the need to act decisively to curtail inflation, even if it entails slowing down economic growth.",
    "Monetary authorities will maintain a hawkish stance as inflation remains well above the target range, risking further imbalances.",
    "A more restrictive policy stance is warranted given the sustained increase in core inflation metrics.",
    "The central bank has signaled that additional rate hikes are likely, reflecting the severity of inflationary concerns.",
    "Policy adjustments are required to ensure that inflation expectations do not become unanchored from long-term targets.",
    "Financial conditions have tightened substantially, and further rate increases are expected to maintain control over inflation.",
    "The central bank's firm commitment to reducing inflation is likely to result in higher interest rates over an extended period.",
    "Given the persistent inflation risks, policymakers have raised the likelihood of higher interest rates to dampen demand.",
    "A sharp reduction in liquidity is anticipated as the central bank prioritizes combating runaway inflation.",
    "Economic growth is expected to decelerate as tighter financial conditions take hold to mitigate inflationary overheating.",
    "Participants agreed that inflationary pressures require continued restrictive measures, including further rate hikes.",
    "The Federal Reserve stands ready to maintain restrictive policies until inflation shows a clear and sustained downward trend.",
    "The risk of delaying further rate hikes could exacerbate inflationary expectations, leading to more significant disruptions.",
    "A prolonged period of higher interest rates is now anticipated, reflecting ongoing inflationary challenges.",
    "The need for swift action to contain inflation has resulted in a more aggressive tightening cycle than previously expected.",
    "Policymakers remain vigilant to the risks of an overheating economy and will act preemptively to stave off inflation.",
    "Monetary tightening is required to restore price stability, even at the expense of slower economic expansion.",
    "The economy's strong fundamentals suggest that further tightening will not derail growth, but will contain inflationary pressures.",
    "A contractionary policy stance is essential to reduce the risk of inflation spiraling out of control.",
    "The persistent rise in inflation rates demands a more forceful response, including higher interest rates and reduced liquidity.",
    "Further monetary tightening is expected to maintain the credibility of the central bank's inflation-targeting framework.",
    "Inflation is projected to remain elevated in the near term, necessitating continued rate hikes.",
    "Policymakers stressed the need to keep inflation expectations anchored through decisive rate adjustments.",
    "The central bank's tightening measures will continue until inflation returns to the target range.",
    "With inflation running well above target, policymakers are prepared to enforce stringent policy measures.",
    "The economy may face short-term disruptions as a result of aggressive rate increases, but these are necessary to restore price stability.",
    "The labor market remains tight, and inflation is rising, reinforcing the need for continued rate hikes.",
    "Participants underscored the importance of sustaining a restrictive policy stance to counter persistent inflationary pressures.",
    "The central bank is committed to a hawkish policy stance to avoid inflation becoming entrenched in the economy.",
    "High levels of inflation have necessitated stronger measures, including successive increases in policy interest rates.",
    "As inflationary pressures mount, the central bank will maintain a vigilant approach, ensuring sufficient monetary tightening.",
    "There is broad consensus that inflation remains a significant concern, justifying further monetary tightening.",
    "The central bank’s swift response to inflationary risks includes raising interest rates faster than initially anticipated.",
    "Economic conditions warrant a continuation of monetary policy tightening to mitigate rising inflation expectations.",
    "The path of interest rates will remain upward until there are clearer signs of inflation decelerating to target levels.",
    "Rapid wage growth and inflationary pressures necessitate prompt action to tighten monetary policy.",
    "The persistence of high inflation has led policymakers to adopt a more hawkish stance, focusing on curbing excess demand.",
    "A restrictive monetary policy stance will remain necessary until inflation shows convincing signs of easing.",
    "As inflation proves more stubborn than expected, the central bank is compelled to maintain a hawkish approach.",
    "With inflation continuing to exceed forecasts, the central bank will implement further rate hikes in the coming months.",
    "Policymakers are unanimous in their assessment that more forceful monetary action is required to control inflation.",
    "Sustained inflationary pressures have triggered a shift towards more restrictive monetary policy measures.",
    "The central bank has reaffirmed its commitment to higher interest rates until inflationary forces are contained.",
    "Given the elevated inflation outlook, tighter financial conditions are necessary to prevent further economic imbalances.",
    "The current inflation trajectory demands a more aggressive tightening cycle to avoid long-term economic distortions.",
    "The central bank will continue to pursue a restrictive policy path to ensure inflation is brought back under control.",
    "Participants expressed concern that delaying rate hikes could result in inflationary expectations becoming unanchored.",
    "The ongoing inflationary environment requires sustained rate hikes to restore balance in the economy.",
    "A series of rate increases is anticipated, given the strength of inflationary pressures across multiple sectors.",
    "Inflationary trends remain a key focus for policymakers, who will act decisively to prevent further acceleration.",
    "The central bank's tightening measures are critical to ensure inflation does not become entrenched.",
    "Inflation has exceeded target levels for an extended period, warranting a more forceful policy response.",
    "Participants highlighted the need for continued monetary tightening to reduce inflationary risks.",
    "Further rate hikes are on the horizon as inflation remains persistent and economic demand robust.",
    "A sustained hawkish stance is required to ensure inflationary expectations remain anchored.",
    "The rapid rise in inflation necessitates further tightening to prevent long-term economic instability.",
    "The central bank’s policy will remain restrictive until inflationary pressures have fully receded.",
    "Persistent inflationary dynamics warrant continued tightening of financial conditions.",
    "Ongoing inflation concerns have led policymakers to increase their"]


In [8]:
dovish_phrases = [
    "Lower interest rates",
    "Rate cuts",
    "Easing monetary policy",
    "Accommodative stance",
    "Stimulate economic growth",
    "Support recovery",
    "Boost demand",
    "Encourage borrowing",
    "Monetary easing",
    "Reduce borrowing costs",
    "Lower inflation expectations",
    "Provide liquidity",
    "Sustainable growth",
    "Supportive monetary conditions",
    "Enhance economic activity",
    "Gradual recovery",
    "Reduce rate pressure",
    "Maintain accommodative policies",
    "Fostering economic stability",
    "Gentle policy adjustments",
    "Prolonged low rates",
    "Increase liquidity",
    "Supportive fiscal measures",
    "Gentle economic conditions",
    "Monetary support",
    "Expansionary fiscal policy",
    "Encourage investment",
    "Facilitate growth",
    "Cautious approach",
    "Willingness to adjust rates",
    "Lowering financial constraints",
    "Favorable monetary conditions",
    "Fostering consumer confidence",
    "Flexibility in policy adjustments",
    "Stable inflation outlook",
    "Financial market support",
    "Promote employment",
    "Support small businesses",
    "Economic resilience",
    "Balanced growth",
    "Encourage economic participation",
    "Mitigate recession risks",
    "Sustained economic recovery",
    "Focus on long-term growth",
    "Consumer support measures",
    "Safeguard economic stability",
    "Policy normalization on hold",
    "Preemptive measures for growth",
    "Gradual policy easing",
    "Focus on sustainable recovery",
    "Dovish guidance",
    "Maintain accommodative environment",
    "Promote stability",
    "Confidence in economic recovery",
    "Encouraging labor market conditions",
    "Proactive support for growth",
    "Interest rate guidance remains low",
    "Fiscal measures to stimulate",
    "Maintain easy credit conditions",
    "Supportive monetary framework",
    "Outlook for sustained growth",
    "Willingness to lower rates",
    "Support for inflation targeting",
    "Adjustments to support economy",
    "Balancing growth and inflation",
    "Economic growth is a priority",
    "Flexibility in rate adjustments",
    "Support for broader recovery",
    "Lower rates to support growth",
    "Avoid abrupt policy changes",
    "Gradual rate normalization",
    "Encouraging economic activity",
    "Prolonged low-rate environment",
    "Focus on economic well-being",
    "Support for long-term stability",
    "Accommodative monetary stance",
    "Cautious monetary policy",
    "Support for market stability",
    "Growth-supportive policies",
    "Promote sustainable investment",
    "Long-term economic outlook",
    "Patient approach to rate adjustments",
    "Willingness to support growth",
    "Stimulus measures in place",
    "Sustained support for growth",
    "Maintain favorable conditions",
    "Stimulate consumer spending",
    "Interest rates likely to stay low",
    "Encouraging growth measures",
    "Supportive central bank policies",
    "Gradual easing of financial conditions",
    "Positive outlook for growth",
    "Optimistic economic forecast",
    "Focused on labor market recovery",
    "Policy adjustments to support economy",
    "Dovish tone in statements",
    "Willingness to provide stimulus",
    "Monitor economic recovery closely",
    "Incentivize economic expansion",
    "Broad support for growth",
    "Gradual improvements in conditions",
    "Proactive policy to support recovery",
    "Strengthen economic foundations",
    "Maintain supportive financial environment",
    "Encouraging signs of recovery",
    "Willing to adjust policy for growth",
    "Sustained fiscal support",
    "Flexibility in economic policy",
    "Accommodative financial conditions",
    "Support for vulnerable sectors",
    "Outlook remains supportive",
    "Commitment to gradual recovery",
    "Easing measures to promote growth",
    "Focus on mitigating risks",
    "Long-term stability prioritized",
    "Gentle approach to monetary easing",
    "Prolonged low-rate environment",
    "Monitor for signs of recovery",
    "Encouraging investment climate",
    "Fiscal stimulus measures in place",
    "Maintaining liquidity in markets",
    "Support for household spending",
    "Interest rates to remain stable",
    "Support for inclusive growth",
    "Flexible monetary policy approach",
    "Patient approach to recovery",
    "Fiscal policy supportive of growth",
    "Positive economic indicators",
    "Willingness to act if necessary",
    "Focus on stable growth",
    "Supportive regulatory environment",
    "Maintain constructive dialogue",
    "Encouraging signs in the economy",
    "Fostering financial stability",
    "Sustained focus on recovery",
    "Support for economic initiatives",
    "Lower rates to bolster growth",
    "Accommodative measures in place",
    "Fiscal measures to boost economy",
    "Broad-based economic support",
    "Patiently monitor economic trends",
    "Dovish sentiment in markets",
    "Maintain a watchful eye on growth",
    "Favorable economic conditions",
    "Continuous support for recovery",
    "Gradual easing of policies",
    "Positive adjustments in guidance",
    "Focus on long-term stability",
    "Supportive economic policies",
    "Encouraging trends in employment",
    "Dovish outlook for interest rates",
    "Monitor for economic signals",
    "Support for overall growth",
    "Support for investment recovery",
    "Fostering economic dynamism",
    "Gradual improvements in labor market",
    "Positive economic sentiment",
    "Avoid tightening too soon",
    "Willing to provide additional support",
    "Long-term growth measures",
    "Maintain conducive conditions",
    "Focus on growth opportunities",
    "Support for consumer confidence",
    "Encouraging environment for growth",
    "Sustained support for recovery efforts",
    "Nurturing economic fundamentals",
    "Maintain focus on economic growth",
    "Balanced approach to recovery",
    "Willing to adjust as needed",
    "Policy flexibility to support recovery",
    "Support for maintaining employment",
    "Easing measures to stimulate growth",
    "Encouraging consumer spending",
    "Proactive measures to boost economy",
    "Dovish adjustments anticipated",
    "Maintain stability in markets",
    "Support for financial conditions",
    "Willingness to adapt policies",
    "Positive outlook for future growth",
    "Strengthen recovery efforts",
    "Supportive measures in place",
    "Monitor labor market conditions",
    "Focus on sustainable policies",
    "Gradual recovery expected",
    "Support for overall economic health",
    "Maintain accommodating approach",
    "Willing to assist struggling sectors",
    "Focus on minimizing risks",
    "Support for vulnerable populations",
    "Positive trajectory in economic data",
    "Nurturing an accommodating environment",
    "Sustained fiscal and monetary support",
    "Lower rates for economic support",
    "Flexibility in monetary policy",
    "Prioritize economic recovery",
    "Stimulate growth through measures",
    "Supportive financial landscape",
    "Nurturing economic well-being",
    "Avoiding drastic changes",
    "Prolonged accommodative measures",
    "Encouraging growth-oriented policies",
    "Proactive support for struggling sectors",
    "Easing measures to stimulate demand",
    "Promoting stability in the economy",
    "Gradual adjustments to support growth",
    "Willingness to change course if needed",
    "Sustained focus on easing measures",
    "Supportive measures for small businesses",
    "Monitor economic conditions closely",
    "Support for inflation targets",
    "Encouraging signs of stability",
    "Maintaining favorable policy environment",
    "Focus on long-term growth objectives",
    "Support for economic resilience",
    "Nurturing growth potential",
    "Focus on consumer sentiment",
    "Commitment to gradual easing",
    "Support for sustainable development",
    "Monitoring trends for adjustments",
    "Willingness to provide further stimulus",
    "Maintain flexibility in policy",
    "Positive indicators for growth",
    "Support for equitable growth",
    "Encouraging market conditions",
    "Maintain an open policy stance",
    "Promote economic participation",
    "Proactive measures to support growth",
    "Willing to adapt to changes",
    "Gentle approach to economic adjustments",
    "Focus on balanced economic recovery",
    "Sustained easing measures",
    "Nurturing a positive growth environment",
    "Positive signals from the market",
    "Supportive regulatory framework",
    "Gradual improvements expected",
    "Maintaining a favorable outlook",
    "Encouraging broader recovery efforts",
    "Sustained attention to economic stability",
    "Focus on restoring consumer confidence",
    "Long-term commitment to growth",
    "Support for inclusive economic policies",
    "Monitoring inflation closely",
    "Support for healthy economic conditions",
    "Gradual approach to adjustments",
    "Willingness to support job growth",
    "Focus on long-term recovery goals",
    "Sustained monetary support",
    "Positive developments in growth",
    "Maintain supportive measures",
    "Encouraging environment for investment",
    "Support for economic initiatives",
    "Commitment to gradual policy adjustments",
    "Monitoring for signs of recovery",
    "Long-term strategy for growth",
    "Support for housing market stability",
    "Flexible policies for economic support",
    "Nurturing a stable economic environment",
    "Focus on fiscal measures for growth",
    "Support for a balanced recovery",
    "Encouraging conditions for growth",
    "Proactive measures to ensure stability",
    "Maintain a constructive approach"]

In [9]:
# Generate embeddings for hawkish and dovish phrases
# Convert the list of embeddings into a NumPy array for each phrase type

hawkish_phrase_embeddings = np.array([generate_text_embedding(phrase)[0] for phrase in hawkish_phrases])
dovish_phrase_embeddings = np.array([generate_text_embedding(phrase)[0] for phrase in dovish_phrases])

In [17]:
hawkish_phrase_embeddings[0][:5]

array([-0.78721964, -0.35772267, -0.26025262, -0.5293739 ,  0.6456497 ],
      dtype=float32)

### Create function that computes cosine similarity

In [12]:
# Function to calculate cosine similarity between two sets of embeddings
def compute_cosine_similarity(embedding_set_1, embedding_set_2):
    # Calculate pairwise cosine similarities between the two embedding sets
    similarity_scores = cosine_similarity(embedding_set_1, embedding_set_2)
    
    return similarity_scores


Use cosine similarity to embedded texts

In [14]:
# Iterate over each row in the speech data to calculate similarity scores for hawkish and dovish embeddings
for idx, speech_row in speech_data.iterrows():
    # Generate the embedding for the speech text
    speech_embedding = generate_text_embedding(speech_row['text'])
    
    # Calculate the mean similarity with hawkish and dovish embeddings
    hawkish_similarity_score = compute_cosine_similarity(speech_embedding, hawkish_phrase_embeddings).mean()
    dovish_similarity_score = compute_cosine_similarity(speech_embedding, dovish_phrase_embeddings).mean()
    
    # Assign similarity scores to the new DataFrame
    speech_data.at[idx, 'hawkish_similarity'] = hawkish_similarity_score
    speech_data.at[idx, 'dovish_similarity'] = dovish_similarity_score

In [15]:
speech_data.head()

,date,text,hawkish_similarity,dovish_similarity
0,2012-01-06,"It is the start of a new year, the traditional...",0.493903,0.446385
1,2012-01-06,\r\n Thank you for the opportunity to sp...,0.457993,0.418316
2,2012-01-07,\r\n Thank you and happy New Year. It is...,0.435671,0.370270
3,2012-01-13,\r\n It's a pleasure to be here this mor...,0.495637,0.421677
4,2012-01-16,\r\n It is certainly a pleasure to be he...,0.356629,0.327922


Save dataframe to the csv file

In [16]:
speech_data.to_csv('df_min_press_speech_factor_similarity.csv', index=False)